
- 법률/규제 리스크는 현재 기술력에서 미래에 문제가 될만한 거니까 이거 노드 위치 변경하는게 좋을듯
- 그리고 웹 검색 필수로 하는게 좋을 것 같은데 고민

In [2]:
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_opentutorial.rag.pdf import PDFRetrievalChain
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Sequence, TypedDict, Literal, List, Dict
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import tools_condition
from langchain_teddynote.models import get_model_name, LLMs
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
import asyncio
import os

In [37]:
class AgentState(TypedDict):

    industry: str
    region: str

    startup_list: List[str]                   # 스타트업 탐색 에이전트가 생성하는 주요 기업명 목록
    startup_profiles: Dict[str, Dict]         # 스타트업별 정보 종합 저장소
    tech_summary : Dict[str, str]              # 각 스타트업 기술 요약 정보
    founder_reputation : Dict[str, Dict]       # 창업자 이력 + 평판 정보
    market_analysis: Dict[str, Dict]          # 시장성 평가 결과
    legal_risk: Dict[str, str]                # 법적/규제 이슈 요약
    competitor_info: Dict[str, Dict]          # 경쟁사 비교 분석
    investment_decision: Dict[str, str]       # 투자 판단 (투자 / 보류 + 사유)
    final_report: str                          # 보고서 생성 에이전트의 출력물 (PDF or Text)

In [50]:
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_opentutorial.rag.pdf import PDFRetrievalChain
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Sequence, TypedDict, Literal
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import tools_condition
from langchain_teddynote.models import get_model_name, LLMs
from langgraph.graph import END, StateGraph, START
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_teddynote.messages import stream_graph
import asyncio
import os

load_dotenv()
logging.langsmith("CH15-Agentic-RAG-Legal")

# 법적 리스크 분석 상태 정의
class LegalRiskAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    company: str
    industry: str
    region: str

    legal_assessments: Dict[str, str]


# 모델 이름 설정
MODEL_NAME = get_model_name(LLMs.GPT4)

# PDF 파일로부터 검색 체인 생성
def create_pdf_retriever():
    file_path = ["data/2023 국내외 AI 규제 및 정책 동향.pdf", "data/인공지능(AI) 관련 국내외 법제 동향.pdf"]
    pdf_file = PDFRetrievalChain(file_path).create_chain()
    pdf_retriever = pdf_file.retriever
    
    # PDF 문서를 기반으로 검색 도구 생성
    retriever_tool = create_retriever_tool(
        pdf_retriever,
        "legal_pdf_retriever",
        "Search and return information about AI legal and regulatory frameworks from the PDF files. They contain essential information on AI regulations, policies, and legal trends relevant for AI startups. The documents are focused on both domestic and international AI legal frameworks.",
        document_prompt=PromptTemplate.from_template(
            "<document><context>{page_content}</context><metadata><source>{source}</source><page>{page}</page></metadata></document>"
        ),
    )
    
    return retriever_tool

# 데이터 모델 정의
class grade(BaseModel):
    """A binary score for relevance checks"""
    binary_score: str = Field(
        description="Response 'yes' if the document is relevant to the legal/regulatory question or 'no' if it is not."
    )

# 문서 관련성 평가 함수 (조건부 엣지에서 사용)
def grade_documents(state: LegalRiskAgentState) -> str:
    # LLM 모델 초기화
    model = ChatOpenAI(temperature=0, model=MODEL_NAME, streaming=True)

    # 구조화된 출력을 위한 LLM 설정
    llm_with_tool = model.with_structured_output(grade)

    # 프롬프트 템플릿 정의
    prompt = PromptTemplate(
        template="""You are a legal expert assessing relevance of a retrieved document to an AI startup's legal/regulatory risk question. \n 
        Here is the retrieved document: \n\n {context} \n\n
        Here is the question about AI legal/regulatory risks: {question} \n
        If the document contains keyword(s) or semantic meaning related to the legal/regulatory question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # llm + tool 바인딩 체인 생성
    chain = prompt | llm_with_tool

    # 현재 상태에서 메시지 추출
    messages = state["messages"]
    last_message = messages[-1]
    question = messages[0].content

    # 검색된 문서 추출
    retrieved_docs = last_message.content

    # 관련성 평가 실행
    scored_result = chain.invoke({"question": question, "context": retrieved_docs})

    # 관련성 여부 추출
    score = scored_result.binary_score

    # 관련성 여부에 따른 결정
    if score == "yes":
        return "generate"
    else:
        print(score)
        return "rewrite"

# 초기 질의 처리 노드
def initial_query(state: LegalRiskAgentState):
    print("\n🟢 [initial_query] 기업 정보 기반 질문 생성 중...")
    question = f"{state['company']}은(는) {state['industry']} 분야 AI 스타트업으로, 관련 법적/규제 리스크는 무엇인가?"
    print(f"➤ 생성된 질문: {question}")
    return {"messages": [HumanMessage(content=question)]}

# 문서 검색 노드 
def pdf_retrieval(state: LegalRiskAgentState):
    print("\n📄 [pdf_retrieval] PDF 기반 법률 문서 검색 시작")
    messages = state["messages"]
    question = messages[-1].content
    retriever_tool = create_pdf_retriever()
    results = retriever_tool.invoke({"query": question})
    print("✅ 검색 완료 - 관련 문서 요약 반환")
    return {"messages": [HumanMessage(content=results)]}

# 질의 재작성 노드
def rewrite(state: LegalRiskAgentState):
    print("\n✍️ [rewrite] 법적 질문이 문서와 관련 없으므로 질의 재작성 시작")
    # 현재 상태에서 메시지 추출
    messages = state["messages"]
    # 원래 질문 추출
    question = messages[0].content
    company = state["company"]
    industry = state["industry"]

    # 질문 개선을 위한 프롬프트 구성
    msg = [
        HumanMessage(
            content=f""" \n 
    Look at the input question about AI legal/regulatory risks for {company} in the {industry} industry and try to reason about the underlying semantic intent / meaning. \n 
    Here is the initial question:
    \n ------- \n
    {question} 
    \n ------- \n
    Formulate an improved question that focuses on specific legal/regulatory frameworks, compliance requirements, or potential legal risks for this AI startup: """,
        )
    ]

    # LLM 모델로 질문 개선
    model = ChatOpenAI(temperature=0, model=MODEL_NAME, streaming=True)
    # Query-Transform 체인 실행
    response = model.invoke(msg)

    # 재작성된 질문 반환
    print(f"🆕 재작성된 질문: {response.content.strip()[:100]}...")
    return {"messages": [response]}

# Web Search 노드 
def web_search(state: LegalRiskAgentState):
    print("\n🌐 [web_search] 웹 기반 보조 법률 정보 검색 시작")
    tavily_tool = TavilySearch()
    
    # 수정된 부분: messages에서 내용 추출
    messages = state["messages"]
    search_query = messages[-1].content
    
    company = state["company"]
    industry = state["industry"]
    
    # 검색 쿼리에 기업 정보 추가
    enhanced_query = f"{search_query} {company} {industry} AI 스타트업 법적 규제"

    search_result = tavily_tool.search(
        query=enhanced_query,  # 검색 쿼리
        topic="legal",     # 법률 주제로 변경
        max_results=3,       # 최대 검색 결과
        format_output=True,  # 결과 포맷팅
    )
    print("✅ 웹 검색 완료 - 요약 내용 반환")
    return {"messages": [HumanMessage(content=search_result)]}



# 법적 분석 노드
def analyze(state: LegalRiskAgentState):
    print("\n🧠 [analyze] 문서 기반 법적 리스크 분석 실행")
    # 현재 상태에서 메시지 추출
    messages = state["messages"]
    # 원래 질문 추출
    question = messages[0].content
    company = state["company"]
    industry = state["industry"]

    # 가장 마지막 메시지 추출 (검색 결과)
    docs = messages[-1].content
    
    # 디버그 메시지 추가
    print(f"기업: {company}")
    print(f"산업: {industry}")
    print(f"질문: {question}")
    print(f"문서 길이: {len(docs) if docs else 0}자")

    # RAG 프롬프트 템플릿 정의
    prompt = PromptTemplate(
        template="""You are a legal expert specialized in AI regulations and policies for startups. 
        Use the following pieces of context to answer the question at the end about {company} in the {industry} industry. 
        If you don't know the answer, just say you don't know. 
        Don't try to make up an answer.
        
        Always structure your response in the following format: (반드시 한국어로 답변)
        1. Legal/Regulatory Analysis: Provide a concise analysis of the legal and regulatory considerations.
        2. Potential Risks: Outline specific risks the AI startup might face.
        3. Compliance Recommendations: Suggest practical steps for ensuring compliance.
        4. International Considerations: Briefly mention any relevant international frameworks if applicable.
        
        {context}
        
        Question: {question}
        
        Helpful Answer:""",
        input_variables=["context", "question", "company", "industry"],
    )

    # LLM 모델 초기화
    llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0, streaming=True)

    # RAG 체인 구성
    rag_chain = prompt | llm | StrOutputParser()

    try:
        # 답변 생성 실행
        response = rag_chain.invoke({
            "context": docs, 
            "question": question,
            "company": company,
            "industry": industry
        })
        print("✅ 분석 완료 - 요약 보고 생성")
        return {"messages": [HumanMessage(content=response)]}
    
    except Exception as e:
        print(f"오류 발생: {e}")
        error_msg = f"응답 생성 중 오류가 발생했습니다: {str(e)}"
        return {"messages": [HumanMessage(content=error_msg)]}

# 법적/규제 리스크 분석 결과 처리 노드
def analyze_legal_risks(state: LegalRiskAgentState):
    print("\n📊 [analyze_legal_risks] 최종 평가 내용 저장")
    company = state["company"]
    messages = state["messages"]
    legal_assessment = messages[-1].content
    
    print(f"📝 저장된 평가: {legal_assessment[:100]}...")
    return {
        "legal_assessments": {company: legal_assessment}
    }

# Agentic RAG를 사용한 법적/규제 리스크 분석 그래프 생성
def create_legal_risk_graph():
    """법적 리스크 분석을 위한 LangGraph 워크플로우 생성 함수"""
    
    # 그래프 정의
    workflow = StateGraph(LegalRiskAgentState)

    # 노드 정의
    workflow.add_node("initial_query", initial_query)  # 초기 질의 처리
    workflow.add_node("pdf_retrieval", pdf_retrieval)  # PDF 문서 검색
    workflow.add_node("rewrite", rewrite)             # 질의 재작성
    workflow.add_node("web_search", web_search)       # 웹 검색
    workflow.add_node("analyze", analyze)             # 법적 분석
    workflow.add_node("legal_risks", analyze_legal_risks)  # 분석 결과 처리

    # 엣지 정의
    workflow.add_edge(START, "initial_query")
    workflow.add_edge("initial_query", "pdf_retrieval")

    # 조건부 엣지: 검색 결과 관련성에 따라 분기
    workflow.add_conditional_edges(
        "pdf_retrieval",
        grade_documents,
        {
            "generate": "analyze",      # 관련성 있으면 분석
            "rewrite": "rewrite"        # 관련성 없으면 재작성
        }
    )

    # 재작성 후 웹 검색 진행
    workflow.add_edge("rewrite", "web_search")
    
    # 웹 검색 후 분석 진행
    workflow.add_edge("web_search", "analyze")
    
    # 분석 결과 처리 및 종료
    workflow.add_edge("analyze", "legal_risks")
    workflow.add_edge("legal_risks", END)

    # 그래프 컴파일 및 반환
    return workflow.compile()


LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agentic-RAG-Legal


In [39]:
sample_state: AgentState = {
    "industry": "AI 의료",
    "region": "대한민국",

    "startup_list": ["AI헬스케어", "닥터AI"],
    
    "startup_profiles": {
        "AI헬스케어": {"설립연도": 2020, "직원 수": 15},
        "닥터AI": {"설립연도": 2021, "직원 수": 10}
    },

    "tech_summary": {
        "AI헬스케어": "딥러닝 기반 영상 진단 솔루션",
        "닥터AI": "AI 기반 처방 최적화 API"
    },

    "founder_reputation": {
        "AI헬스케어": {"이름": "홍길동", "이력": "서울대 의대 → KAIST AI Lab", "리스크": "없음"},
        "닥터AI": {"이름": "김영희", "이력": "IBM Watson Health", "리스크": "전직 투자 관련 분쟁 이력 있음"}
    },

    "market_analysis": {
        "AI헬스케어": {"시장 규모": "500억", "경쟁 강도": "중간"},
        "닥터AI": {"시장 규모": "300억", "경쟁 강도": "높음"}
    },

    "legal_risk": {
        "AI헬스케어": "의료 데이터 비식별화 이슈 존재",
        "닥터AI": "처방 시스템의 인증 요건 미충족"
    },

    "competitor_info": {
        "AI헬스케어": {"주요 경쟁사": ["메디에이아이", "딥메드"], "차별점": "국내 유일 영상 기반 진단"},
        "닥터AI": {"주요 경쟁사": ["헬로닥"], "차별점": "처방 추천에 강화학습 사용"}
    },

    "investment_decision": {
        "AI헬스케어": "투자 - 기술 우수성과 규제 적합성 확보",
        "닥터AI": "보류 - 법적 이슈 추가 검토 필요"
    },

    "final_report": "[보고서 미생성]"  # 이후 보고서 생성 노드에서 갱신
}

In [51]:
async def legal_risk_analysis_agent(company, industry, region):
    # 법적/규제 리스크 그래프 생성
    legal_graph = create_legal_risk_graph()

    # 초기 상태 설정 (legal_assessments 필드 추가)
    initial_state: LegalRiskAgentState = {
        "messages": [HumanMessage(content=f"{company}의 법적 규제 분석")],
        "company": company,
        "industry": industry,
        "region": region,
        "legal_assessments": {}  # 빈 딕셔너리로 초기화
    }

    # 그래프 실행
    result = await legal_graph.ainvoke(initial_state)

    # 결과 반환 (상위 시스템에서 사용할 수 있도록)
    if "legal_assessments" in result and company in result["legal_assessments"]:
        return result["legal_assessments"][company]
    else:
        return "법적 평가를 완료하지 못했습니다."

await legal_risk_analysis_agent("닥터AI", "의료 AI", "글로벌")


🟢 [initial_query] 기업 정보 기반 질문 생성 중...
➤ 생성된 질문: 닥터AI은(는) 의료 AI 분야 AI 스타트업으로, 관련 법적/규제 리스크는 무엇인가?

📄 [pdf_retrieval] PDF 기반 법률 문서 검색 시작
✅ 검색 완료 - 관련 문서 요약 반환

🧠 [analyze] 문서 기반 법적 리스크 분석 실행
기업: 닥터AI
산업: 의료 AI
질문: 닥터AI의 법적 규제 분석
문서 길이: 3944자
✅ 분석 완료 - 요약 보고 생성

📊 [analyze_legal_risks] 최종 평가 내용 저장
📝 저장된 평가: 1. 법적/규제 분석: 닥터AI는 의료 AI 산업에 속하므로, 고위험 AI 시스템으로 분류될 가능성이 높습니다. 이에 따라 EU 인공지능법 및 국내 관련 법령에 따라 시장 출시 전...


'1. 법적/규제 분석: 닥터AI는 의료 AI 산업에 속하므로, 고위험 AI 시스템으로 분류될 가능성이 높습니다. 이에 따라 EU 인공지능법 및 국내 관련 법령에 따라 시장 출시 전 위험성 평가 및 출시 후 모니터링 의무가 부과될 수 있습니다. 또한, 개인정보 보호와 관련된 법률을 준수해야 하며, AI의 결과물에 대한 성실의무를 이행해야 합니다.\n\n2. 잠재적 위험: 닥터AI는 개인정보 유출, 오작동으로 인한 안전사고, 그리고 AI의 결과물에 대한 법적 책임 문제 등 다양한 위험에 직면할 수 있습니다. 특히, 의료 분야에서의 잘못된 진단이나 치료 권고는 심각한 법적 문제를 초래할 수 있습니다.\n\n3. 준수 권장 사항: 닥터AI는 다음과 같은 조치를 취해야 합니다. 첫째, 고위험 AI 시스템으로서의 요구사항을 충족하기 위한 위험 관리 체계를 구축해야 합니다. 둘째, 개인정보 보호 관련 법률을 준수하기 위해 데이터 수집 및 처리 방침을 명확히 하고, 사용자에게 투명하게 고지해야 합니다. 셋째, AI의 결과물에 대한 정확성을 검토하고, 이를 문서화하여 법적 책임을 최소화해야 합니다.\n\n4. 국제적 고려사항: EU 인공지능법을 포함한 국제적인 AI 규제 프레임워크는 닥터AI의 운영에 중요한 영향을 미칠 수 있습니다. 특히, EU 시장에 진출할 경우 해당 법률을 준수해야 하며, 국제적인 윤리 기준과 신뢰성 기준을 따르는 것이 중요합니다.'